In [12]:
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
import os
from langgraph.prebuilt import tool_node
import google.generativeai as genai
import fitz
from typing import TypedDict
from pprint import pprint
import json

In [13]:
class AgentType(TypedDict):
    offer_analysis: dict
    preference_alignment: dict
    acceptance_prediction: dict

In [14]:
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")

In [15]:

def acceptance_likelihood_agent(state):
    # Extract and parse the previous outputs
    offer = state["offer_analysis"]
    preference = state["preference_alignment"]

    # Ensure data is parsed to dict if it's still a JSON string
    if isinstance(offer, str):
        offer = json.loads(offer)
    if isinstance(preference, str):
        preference = json.loads(preference)

    # Create a prompt for Gemini
    prompt = f"""
    You are an AI placement counselor.
    Based on the offer details and the candidate's preferences, predict the likelihood that the student will accept the job.

    Offer Details:
    {json.dumps(offer, indent=2)}

    Candidate Preferences:
    {json.dumps(preference, indent=2)}

    Respond in the following format:
    {{
      "acceptance_likelihood": <percentage from 0 to 100>,
      "reasoning": "short explanation of why the student is likely/unlikely to accept"
    }}
    """

    response = model.generate_content(prompt)
    return {"acceptance_prediction": response.text}

In [16]:
builder = StateGraph(AgentType)


builder.add_node("AcceptanceLikelihood", acceptance_likelihood_agent)
builder.set_entry_point("AcceptanceLikelihood")
builder.set_finish_point("AcceptanceLikelihood")

graph = builder.compile()


In [17]:

result = graph.invoke({
    "offer_analysis": {'bond_terms': None,
 'job_title': 'Software Engineer',
 'joining_date': '2017-05-18',
 'location': 'Nasik',
 'salary': {'gross_annual': 314935.38,
            'monthly_breakdown': {'basic': 8000,
                                  'bonus': 6300,
                                  'conveyance': 3600,
                                  'education_allowance': 3600,
                                  'gratuity': 4615,
                                  'hra': 4800,
                                  'leave_benefit': 14000,
                                  'medical_reimbursement': 1500,
                                  'mediclaim': 1000,
                                  'pf': 960,
                                  'variable': 3000},
            'variable_component': 'Based on monthly PPI (Personal Performance '
                                  'Index) + CPI (Company Performance Index)'}},
    "preference_alignment": {'career_goals': 'Contribute to innovative projects that make a difference',
 'expected_ctc': None,
 'interests': ['Full Stack Development',
               'MERN Stack',
               'PERN Stack',
               'Scalable Web Applications',
               'Innovative Projects'],
 'preferred_location': 'Coimbatore, Tamil Nadu, India',
 'skills': ['JavaScript',
            'Nodejs',
            'fastAPI',
            'Python',
            'Express js',
            'MongoDB',
            'PostgreSQL',
            'Redis',
            'Elasticsearch',
            'RabbitMQ',
            'React js',
            'Next js',
            'Amazon S3',
            'AWS Elastic Beanstalk',
            'AWS Cloudfront',
            'Prisma ORM',
            'UI/UX',
            'Responsive Web Design']}
})

json_string = result["acceptance_prediction"].strip().replace('```json\n', '').replace('\n```', '')
formatted_dict = json.loads(json_string)
pprint(formatted_dict)


{'acceptance_likelihood': 30,
 'reasoning': "The candidate's strong preference for Coimbatore significantly "
              'lowers the likelihood of acceptance.  The Nasik location is a '
              "major drawback. While the salary is reasonable,  there's no "
              "information on the specifics of the projects or the company's "
              "culture, both of which are important given the candidate's "
              "focus on innovative projects. The candidate's skills align with "
              'the Software Engineer role, but the location mismatch is the '
              'dominant factor in predicting a low probability of acceptance.'}
